<span style="color:red; font-family:Helvetica Neue, Helvetica, Arial, sans-serif; font-size:2em;">An Exception was encountered at '<a href="#papermill-error-cell">In [1]</a>'.</span>

# Futures long-only performance summary

This notebook summarises long-only performance statistics for major futures contracts provided by various data sources. Daily returns are computed by rolling the front contract before either the first notice day or the last trade day to avoid deliveries. Concretely, a daily return on a given day is written as follows:

$$
r_{t, i} = \frac{F_{t, i} - F_{t-1, t}}{F_{rolldate, i}}
$$

where $F_{t, i}$ is the futures price at time $t$ for an instrument $i$, $F_{rolldate, i}$ is the futures price when we roll into the contract. All functionalities as well as underlying data are provided by a private library `vivace`.

<span id="papermill-error-cell" style="color:red; font-family:Helvetica Neue, Helvetica, Arial, sans-serif; font-size:2em;">Execution using papermill encountered an exception here and stopped:</span>

In [1]:
%matplotlib inline
from datetime import datetime
import logging
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

import pandas as pd
import matplotlib.pyplot as plt
plt.style.use('bmh')

from vivace.backtest.contract import all_futures_lo_map
from vivace.backtest.engine import BacktestEngine
from vivace.backtest.enums import Strategy
from vivace.backtest.stats import Performance, PerfStats

ReadTimeout: HTTPSConnectionPool(host='api.binance.com', port=443): Read timed out. (read timeout=10)

`all_futures_lo_map` contains the list of all available futures contracts for long-only calculation.

In [ ]:
all_futures_lo_map

Without any signals, `BacktestEngine` calculates simple long-only strategies by default. `Performance` computes typical performance metrics such as Sharpe ratio.

In [ ]:
results = []

for ticker, name in all_futures_lo_map.items():
    engine = BacktestEngine(
        instrument=ticker,
        strategy=Strategy.DELTA_ONE.value,
        log_level=logging.WARNING,
    )
    engine.run()
    equity_curve = engine.calculate_equity_curve(calculate_net=False)
    stats = Performance(equity_curve).summary().squeeze()
    
    results.append([
        name,
        equity_curve.index[0].strftime('%d-%b-%Y'),
        equity_curve.index[-1].strftime('%d-%b-%Y'),
        stats[PerfStats.CAGR.value],
        stats[PerfStats.ANN_VOL.value],
        stats[PerfStats.SHARPE.value],
        stats[PerfStats.SKEWNESS.value],
    ])
    
result = pd.DataFrame(results, columns=[
    'contract', 'start_end', 'end_date',
    PerfStats.CAGR.value, 
    PerfStats.ANN_VOL.value,
    PerfStats.SHARPE.value,
    PerfStats.SKEWNESS.value,
]).set_index('contract')

In [ ]:
result.shape

# Performance summary

This table displays contract information such as the contract name as well as performance statistics. Bar charts are superimposed for annualised returns and Sharpe ratios based on its value.   Statistics are in contracts' local currencies.
Note that some start dates do not match the exact inception of the futures contract because of limited data availability.

In [ ]:
(result
 .style.format({
        PerfStats.CAGR.value: "{:.1%}",
        PerfStats.ANN_VOL.value: "{:.1%}",
        PerfStats.SHARPE.value: "{:.2f}",
        PerfStats.SKEWNESS.value: "{:.2f}",
    })
 .bar(subset=PerfStats.CAGR.value)
 .bar(subset=PerfStats.SHARPE.value))

In [ ]:
fig, ax = plt.subplots(figsize=(8, 20))
ax.set_title("Long-only Sharpe ratio")
ax.set_ylabel("Sharpe ratio")
result[PerfStats.SHARPE.value].loc[::-1].plot(kind="barh", ax=ax, grid=True);  # reverse the plot order
ax.axvline(0, lw=1);

# Plot cumulative return series

Here only the E-mini S&P 500 futures is shown as an example.

In [ ]:
fig, ax = plt.subplots(figsize=(8, 5))
engine = BacktestEngine(
    instrument='ES',
    strategy=Strategy.DELTA_ONE.value,
    log_level=logging.WARNING,
)
engine.run()
engine.calculate_equity_curve(calculate_net=False).plot(ax=ax, logy=True)
ax.set_ylabel('Cumulative returns');

In [ ]:
print(f'Updated: {datetime.utcnow().strftime("%d-%b-%Y %H:%M")}')